In [1]:
!wget https://www.dropbox.com/s/x0vjwjeuaawo6yr/entitytyping.zip?dl=0  -O entitytyping.zip
!unzip entitytyping.zip

--2022-05-29 05:45:43--  https://www.dropbox.com/s/x0vjwjeuaawo6yr/entitytyping.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/x0vjwjeuaawo6yr/entitytyping.zip [following]
--2022-05-29 05:45:43--  https://www.dropbox.com/s/raw/x0vjwjeuaawo6yr/entitytyping.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf4fac902d7053de2b87abfed8d.dl.dropboxusercontent.com/cd/0/inline/BmLBF44L1I8yAEjPirhj_6J-bjcSmm5XSLh_9Y42LeLVImPnEbzxvamu2xV9kVb81G-NPZb88OmlkuNGWjRS3x5kWJ0chwJ3LV82KozqDGC2nggMZ0oeZS-1h2gvyuv5vvVtNJC2ajngwY5mGqvQxpUcL1WdYvRGo1NX7fU42TNDgQ/file# [following]
--2022-05-29 05:45:43--  https://ucf4fac902d7053de2b87abfed8d.dl.dropboxusercontent.com/cd/0/inline/BmLBF44L1I8yAEjPirhj_6J-bjcSmm5XSLh_9Y42LeLVImPnEbz

In [1]:
!pip install openprompt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 146 kB 8.5 MB/s 
     |████████████████████████████████| 346 kB 47.5 MB/s 
     |████████████████████████████████| 4.2 MB 45.9 MB/s 
     |████████████████████████████████| 125 kB 52.3 MB/s 
     |████████████████████████████████| 1.2 MB 44.4 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 38.7 MB/s 
     |████████████████████████████████| 596 kB 41.5 MB/s 
     |████████████████████████████████| 212 kB 32.2 MB/s 
     |████████████████████████████████| 140 kB 52.1 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 1.1 MB 42.8 MB/s 
     |████████████████████████████████| 127 kB 54.0 MB/s 
     |████████████████████████████████| 271 kB 54.2 MB/s 
     |████████████████████████████████| 144 kB 55.7 MB/s 
     |██████████████████████████████

In [11]:
import os

import torch
from openprompt.data_utils.typing_dataset import PROCESSORS
import argparse

from openprompt.prompts import ManualTemplate, ManualVerbalizer
from tqdm import tqdm

parser = argparse.ArgumentParser("")

parser.add_argument("--shot", type=int, default=5)
parser.add_argument("--seed", type=int, default=144)

parser.add_argument("--plm_eval_mode", action="store_true")
parser.add_argument("--model", type=str, default='roberta')
parser.add_argument("--model_name_or_path", default='roberta-large')
parser.add_argument("--result_file", type=str, default="sfs_scripts/results_fewshot_manual_kpt.txt")
parser.add_argument("--openprompt_path", type=str, default="datasets")

parser.add_argument("--verbalizer", default='kpt',type=str)
parser.add_argument("--calibration", default='--calibration',action="store_true")
parser.add_argument("--nocut", action="store_true")
parser.add_argument("--filter", default="tfidf_filter", type=str)
parser.add_argument("--template_id", default=1,type=int)
parser.add_argument("--max_token_split", default=-1, type=int)
parser.add_argument("--dataset",default='agnews',type=str)
parser.add_argument("--write_filter_record", action="store_true")
args = parser.parse_args(args=[])

# base_path = "Few-Nerd"
#
dataset_name = "FewNERD"
# dataset_path = os.path.join(base_path, dataset_name)
processor = PROCESSORS[dataset_name.lower()]()
dataset_path='Few-Nerd'
dataset={}

train_dataset = processor.get_train_examples(dataset_path)

## Zero-shot

In [12]:
dev_dataset = processor.get_dev_examples(dataset_path)
test_dataset = processor.get_test_examples(dataset_path)
class_labels=processor.get_labels()
scriptsbase = ""
scriptformat = "txt"
cutoff=0.5 if (not args.nocut) else 0.0
max_seq_l = 128
batch_s = 10

dataset['train']=train_dataset
dataset['test']=test_dataset
dataset['support']=dev_dataset

### 加载预训练模型

In [13]:
from openprompt.utils.reproduciblity import set_seed
set_seed()

from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm(args.model, args.model_name_or_path)

mytemplate = ManualTemplate(tokenizer=tokenizer).from_file(f"template.txt", choice=args.template_id)

myverbalizer = ManualVerbalizer(tokenizer, classes=class_labels).from_file(f"verbalizer.{scriptformat}")


### 实体类型分类问题

In [14]:
from openprompt import PromptModel, PromptDataLoader,PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate,
                           verbalizer=myverbalizer,freeze_plm=False, plm_eval_mode=args.plm_eval_mode)

if use_cuda:
    prompt_model=  prompt_model.cuda()


In [15]:
myrecord= ""
if args.write_filter_record:
    record_prefix = "="*20+"\n"
    record_prefix += f"dataset {args.dataset}\t"
    record_prefix += f"temp {args.template_id}\t"
    record_prefix += f"seed {args.seed}\t"
    record_prefix += f"cali {args.calibration}\t"
    record_prefix += f"filt {args.filter}\t"
    record_prefix += "\n"
    myrecord = record_prefix +myrecord
    with open("../sfs_scripts/filter_record_file.txt",'a')  as fout_rec:
        fout_rec.write(myrecord)
    exit()


test_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=max_seq_l, decoder_max_length=3,
    batch_size=batch_s,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="tail")


tokenizing: 96902it [02:30, 642.48it/s]


In [16]:
allpreds = []
alllabels = []
pbar = tqdm(test_dataloader)
for step, inputs in enumerate(pbar):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)#bs*labels_num
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

100%|██████████| 48451/48451 [31:51<00:00, 25.34it/s]


In [18]:
acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
recall=sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(alllabels)
micro_f1=2*acc*recall/(acc+recall)

In [19]:
print(acc,recall,micro_f1)

0.1915233947699738 0.1915233947699738 0.1915233947699738


# few-shot

In [ ]:
train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=max_seq_l, decoder_max_length=3,
    batch_size=batch_s,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="tail")

In [ ]:



validation_dataloader = PromptDataLoader(dataset=dataset["support"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=max_seq_l, decoder_max_length=3,
    batch_size=batch_s,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="tail")



test_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=max_seq_l, decoder_max_length=3,
    batch_size=batch_s,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="tail")
#['In the early 1930s the band moved to the Grill Room of the Taft Hotel in New York ; the band was renamed ``
# George Hall and His Hotel Taft Orchestra `` .', '. In this sentence,', ' Grill Room', ' is a', '<mask>', '.']
from random import random

def evaluate(prompt_model, dataloader, desc):
    prompt_model.eval()
    allpreds = []
    alllabels = []
    pbar = tqdm(dataloader, desc=desc)
    for step, inputs in enumerate(pbar):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        alllabels.extend(labels.cpu().tolist())
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
    return acc











'''

assert processor.get_num_labels() == 66
assert processor.get_labels() == [
    "person-actor", "person-director", "person-artist/author", "person-athlete", "person-politician", "person-scholar", "person-soldier", "person-other",
    "organization-showorganization", "organization-religion", "organization-company", "organization-sportsteam", "organization-education", "organization-government/governmentagency", "organization-media/newspaper", "organization-politicalparty", "organization-sportsleague", "organization-other",
    "location-GPE", "location-road/railway/highway/transit", "location-bodiesofwater", "location-park", "location-mountain", "location-island", "location-other",
    "product-software", "product-food", "product-game", "product-ship", "product-train", "product-airplane", "product-car", "product-weapon", "product-other",
    "building-theater", "building-sportsfacility", "building-airport", "building-hospital", "building-library", "building-hotel", "building-restaurant", "building-other",
    "event-sportsevent", "event-attack/battle/war/militaryconflict", "event-disaster", "event-election", "event-protest", "event-other",
    "art-music", "art-writtenart", "art-film", "art-painting", "art-broadcastprogram", "art-other",
    "other-biologything", "other-chemicalthing", "other-livingthing", "other-astronomything", "other-god", "other-law", "other-award", "other-disease", "other-medical", "other-language", "other-currency", "other-educationaldegree",
]


assert processor.get_num_labels() == 66
assert processor.get_labels() == [
    "person-actor", "person-director", "person-artist/author", "person-athlete", "person-politician", "person-scholar", "person-soldier", "person-other",
    "organization-showorganization", "organization-religion", "organization-company", "organization-sportsteam", "organization-education", "organization-government/governmentagency", "organization-media/newspaper", "organization-politicalparty", "organization-sportsleague", "organization-other",
    "location-GPE", "location-road/railway/highway/transit", "location-bodiesofwater", "location-park", "location-mountain", "location-island", "location-other",
    "product-software", "product-food", "product-game", "product-ship", "product-train", "product-airplane", "product-car", "product-weapon", "product-other",
    "building-theater", "building-sportsfacility", "building-airport", "building-hospital", "building-library", "building-hotel", "building-restaurant", "building-other",
    "event-sportsevent", "event-attack/battle/war/militaryconflict", "event-disaster", "event-election", "event-protest", "event-other",
    "art-music", "art-writtenart", "art-film", "art-painting", "art-broadcastprogram", "art-other",
    "other-biologything", "other-chemicalthing", "other-livingthing", "other-astronomything", "other-god", "other-law", "other-award", "other-disease", "other-medical", "other-language", "other-currency", "other-educationaldegree",
]
assert dev_dataset[0].text_a == "The final stage in the development of the Skyfox was the production of a model with tricycle landing gear to better cater for the pilot training market ."
assert dev_dataset[0].meta["entity"] == "Skyfox"
assert dev_dataset[0].label == 30
'''






In [24]:
import random

In [30]:
this_run_unicode=str(random.randint(0, 1e10))
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
optimizer1 = AdamW(prompt_model.parameters(), lr=3e-5)
tot_loss = 0
log_loss = 0
best_val_acc = 0
max_epochs =5
for epoch in range(max_epochs):
    tot_loss = 0
    prompt_model.train()
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)#bs*class_nums
        labels = inputs['label']#bs*1
        loss = loss_func(logits, labels)
        loss.requires_grad_(True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(prompt_model.parameters(), 1.0)#梯度裁剪
        tot_loss += loss.item()
        optimizer1.step()
        optimizer1.zero_grad()
        if step!=0 and step%200==0:
          print('step={},average loss={}'.format(step,tot_loss/step))

    val_acc = evaluate(prompt_model, validation_dataloader, desc="Valid")
    if val_acc>=best_val_acc:
        torch.save(prompt_model.state_dict(),f"{this_run_unicode}.ckpt")
        best_val_acc = val_acc
    print("Epoch {}, val_acc {}".format(epoch, val_acc), flush=True)

prompt_model.load_state_dict(torch.load(f"{this_run_unicode}.ckpt"))
prompt_model = prompt_model.cuda()
test_acc = evaluate(prompt_model, test_dataloader, desc="Test")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


step=200,average loss=3.3661122047901153
step=400,average loss=3.4840178912878037
step=600,average loss=3.3994029732545217
step=800,average loss=3.387121944874525
step=1000,average loss=3.4056431230306625
step=1200,average loss=3.4003532695770264
step=1400,average loss=3.408081677726337
step=1600,average loss=3.4201036425679923
step=1800,average loss=3.377756201426188
step=2000,average loss=3.364802974998951
step=2200,average loss=3.3527766661210494
step=2400,average loss=3.3474711544811724
step=2600,average loss=3.35005833534094
step=2800,average loss=3.324537343766008
step=3000,average loss=3.3304373717308042
step=3200,average loss=3.3687009147927167
step=3400,average loss=3.4157961751783597
step=3600,average loss=3.454931202828884
step=3800,average loss=3.4877211680224067
step=4000,average loss=3.5181861983835696
step=4200,average loss=3.5420341714507058
step=4400,average loss=3.5632577844912356
step=4600,average loss=3.58414865942105
step=4800,average loss=3.602685950671633
step=50

KeyboardInterrupt: ignored